In [1]:
import os
from google.colab import drive
from google.colab import userdata

drive.mount("/content/drive")

HF_TOKEN = userdata.get("HF_TOKEN")
os.environ["HF_TOKEN"] = HF_TOKEN

print("\nHugging Face Token successfully set.")

OPENAI_API_KEY = userdata.get("OPENAI_API")
os.environ["OPENAI_API"] = OPENAI_API_KEY

print("\nOpenAI API successfully set.\n")

%cd /content/drive/MyDrive/ES-CSA/data/

Mounted at /content/drive

Hugging Face Token successfully set.

OpenAI API successfully set.

/content/drive/MyDrive/ES-CSA/data


In [2]:
import pandas as pd
import sqlite3

In [3]:
# Load Raw User Profile Data

user_profile_path = "raw/UserProfile.csv"

user_profile = pd.read_csv(user_profile_path)

user_profile["user_index"] = user_profile.index

print("Successfully loaded Raw User Profile data.")

Successfully loaded Raw User Profile data.


In [4]:
# Creating SQLite Database for Mapping MSISDN -> User

db_path = "/content/drive/MyDrive/ES-CSA/data/msisdn_mapping.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS msisdn_mapping (
        msisdn TEXT PRIMARY KEY,
        user_name TEXT,
        user_index INTEGER
    )
""")

msisdn_to_user = {}

for idx, row in user_profile.iterrows():
    user_name = row["Name"]
    msisdn = str(row["MSISDN"])
    user_index = row["user_index"]
    msisdn_to_user[msisdn] = {
        "user_name": user_name,
        "user_index": user_index
    }

print("MSISDN-User Mapping:\n")
for k, v in list(msisdn_to_user.items())[:5]:
    print(f"MSISDN: {k} -> {v}")

MSISDN-User Mapping:

MSISDN: 9230610000463 -> {'user_name': 'User 1', 'user_index': 0}
MSISDN: 9230347659110 -> {'user_name': 'User 2', 'user_index': 1}
MSISDN: 9230141002657 -> {'user_name': 'User 3', 'user_index': 2}
MSISDN: 9230162731400 -> {'user_name': 'User 4', 'user_index': 3}
MSISDN: 9230108284824 -> {'user_name': 'User 5', 'user_index': 4}


In [5]:
# Store Records in SQLite Database

cursor.executemany(
    "INSERT OR REPLACE INTO msisdn_mapping (msisdn, user_name, user_index) VALUES (?, ?, ?)",
    [(msisdn, info["user_name"], info["user_index"]) for msisdn, info in msisdn_to_user.items()]
)

conn.commit()
conn.close()

print("MSISDN to User mappings successfully stored in SQLite database.")

MSISDN to User mappings successfully stored in SQLite database.


In [6]:
# Verifying Sample

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT * FROM msisdn_mapping")
records = cursor.fetchall()

# Display results
if records:
    for row in records[:10]:
        print(f"MSISDN: {row[0]} | User Name: {row[1]} | User Index: {row[2]}")
else:
    print("No records found in the database!")

conn.close()

MSISDN: 9230610000463 | User Name: User 1 | User Index: 0
MSISDN: 9230347659110 | User Name: User 2 | User Index: 1
MSISDN: 9230141002657 | User Name: User 3 | User Index: 2
MSISDN: 9230162731400 | User Name: User 4 | User Index: 3
MSISDN: 9230108284824 | User Name: User 5 | User Index: 4
MSISDN: 9230945377575 | User Name: User 6 | User Index: 5
MSISDN: 9230406147934 | User Name: User 7 | User Index: 6
MSISDN: 9230181101498 | User Name: User 8 | User Index: 7
MSISDN: 9230797287920 | User Name: User 9 | User Index: 8
MSISDN: 9230839186102 | User Name: User 10 | User Index: 9
